# Notebook 1 - Preparing the Data


### Importing Libraries and Loading Data

First, we need to import the necessary libraries and load the dataset.

In [1]:
import kagglehub
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = kagglehub.dataset_download("open-source-sports/baseball-databank")

print("Path to dataset files:", path)

Path to dataset files: /Users/willgurley/.cache/kagglehub/datasets/open-source-sports/baseball-databank/versions/2


This dataset is from the [Baseball Databank on Kaggle](https://www.kaggle.com/datasets/open-source-sports/baseball-databank). To get the data, you can use the code above to download it using the `kagglehub` library. If you don't have `kagglehub` installed, you can install it using pip:

```bash
pip install kagglehub
```

Looking at the Baseball-Databank folder, we can see that there is a variety of CSV files containing different types of baseball data. To look at team success vs. payroll, we will primarily be using the `Teams.csv` and `Salaries.csv` files. The following code was used to load these files into pandas dataframes, merge the necessary columns into a single dataframe, and display the first few rows of the merged dataframe, 'df',

In [3]:
# Load the necessary CSV files
teams = pd.read_csv("/Users/willgurley/.cache/kagglehub/datasets/open-source-sports/baseball-databank/versions/2/Teams.csv")
salaries = pd.read_csv("/Users/willgurley/.cache/kagglehub/datasets/open-source-sports/baseball-databank/versions/2/Salaries.csv")

# Filter teams to post-1985 (when salary data becomes available)
teams_post_1985 = teams[teams['yearID'] >= 1985]

# Aggregate salaries by team and year, counting each player once per team-year
# Drop duplicate player entries within the same team and year (playerID, teamID, yearID)
salaries_unique = salaries.drop_duplicates(subset=['playerID','teamID','yearID'])
# Sum salaries per team and year
team_year_salary = salaries_unique.groupby(['teamID','yearID'], as_index=False)['salary'].sum()
# Rename columns for clarity
team_year_salary = team_year_salary.rename(columns={'teamID':'team_id','yearID':'year','salary':'total_salary'})
# Sort values
team_year_salary = team_year_salary.sort_values(['year','team_id']).reset_index(drop=True)

# Compute total wins per team and year from teams_post_1985
wins = teams_post_1985.groupby(['teamID','yearID'], as_index=False)['W'].sum()
wins = wins.rename(columns={'teamID':'team_id','yearID':'year','W':'wins'})

# Merge wins into team_year_salary
merged = team_year_salary.merge(wins, on=['team_id','year'], how='left')
# Fill missing wins with 0 (if a team has salary record but no teams entry)
merged['wins'] = merged['wins'].fillna(0).astype(int)

# Merge playoff series win data from teams dataframe
# Get playoff win columns (DivWin, WCWin, LgWin, WSWin) from teams_post_1985
playoff_wins = teams_post_1985[['teamID', 'yearID', 'DivWin', 'WCWin', 'LgWin', 'WSWin']].copy()
playoff_wins = playoff_wins.rename(columns={'teamID': 'team_id', 'yearID': 'year'})

# Convert playoff win columns to boolean (True if 'Y', False otherwise)
playoff_wins['div_win'] = (playoff_wins['DivWin'] == 'Y')
playoff_wins['wc_win'] = (playoff_wins['WCWin'] == 'Y')
playoff_wins['lg_win'] = (playoff_wins['LgWin'] == 'Y')
playoff_wins['ws_win'] = (playoff_wins['WSWin'] == 'Y')

# Get total playoff games won from SeriesPost.csv
series_post = pd.read_csv("/Users/willgurley/.cache/kagglehub/datasets/open-source-sports/baseball-databank/versions/2/SeriesPost.csv")
series_post = series_post[series_post['yearID'] >= 1985]

# Calculate playoff games won by winners (wins column)
winner_games = series_post.groupby(['yearID', 'teamIDwinner'], as_index=False)['wins'].sum()
winner_games = winner_games.rename(columns={'yearID': 'year', 'teamIDwinner': 'team_id', 'wins': 'games_won'})

# Calculate playoff games won by losers (losses column - this is games the loser won)
loser_games = series_post.groupby(['yearID', 'teamIDloser'], as_index=False)['losses'].sum()
loser_games = loser_games.rename(columns={'yearID': 'year', 'teamIDloser': 'team_id', 'losses': 'games_won'})

# Combine both and sum total playoff games won per team per year
all_playoff_games = pd.concat([winner_games, loser_games], ignore_index=True)
total_playoff_games = all_playoff_games.groupby(['year', 'team_id'], as_index=False)['games_won'].sum()
total_playoff_games = total_playoff_games.rename(columns={'games_won': 'total_playoff_wins'})

# Merge with playoff_wins
playoff_wins = playoff_wins.merge(total_playoff_games, on=['team_id', 'year'], how='left')
playoff_wins['total_playoff_wins'] = playoff_wins['total_playoff_wins'].fillna(0).astype(int)

# Merge with merged dataframe
merged = merged.merge(playoff_wins[['team_id', 'year', 'div_win', 'wc_win', 'lg_win', 'ws_win', 'total_playoff_wins']], 
                      on=['team_id', 'year'], 
                      how='left')

# Fill NaN values (teams that didn't make playoffs)
merged['div_win'] = merged['div_win'].fillna(False)
merged['wc_win'] = merged['wc_win'].fillna(False)
merged['lg_win'] = merged['lg_win'].fillna(False)
merged['ws_win'] = merged['ws_win'].fillna(False)
merged['total_playoff_wins'] = merged['total_playoff_wins'].fillna(0).astype(int)

# Create df (same as merged at this point)
df = merged.copy()

df.head()


,team_id,year,total_salary,wins,div_win,wc_win,lg_win,ws_win,total_playoff_wins
0,ATL,1985,14807000,66,False,False,False,False,0
1,BAL,1985,11560712,83,False,False,False,False,0
2,BOS,1985,10897560,81,False,False,False,False,0
3,CAL,1985,14427894,90,False,False,False,False,0
4,CHA,1985,9846178,85,False,False,False,False,0


### About the Data

The data was produced by the Sean Lahman Baseball Database, which is a comprehensive database of Major League Baseball statistics and history. The dataset includes information on players, teams, games, and various statistics from the inception of professional baseball to the present day. To produce the data, Sean Lahman compiled and digitized historical baseball records from various sources, including official MLB records, newspapers, and other archival materials. The database is widely used by researchers, analysts, and baseball enthusiasts for statistical analysis and historical research.



Let's take a look at the dataframe's features:

In [4]:
COLS = pd.DataFrame({
    'column_name': df.columns,
    'data_type': df.dtypes.astype(str),
    'scale': [
        'nominal',      # team_id - categorical, no order
        'interval',     # year - numeric with equal intervals
        'ratio',        # total_salary - numeric with true zero
        'ratio',        # wins - numeric with true zero
        'nominal',      # div_win - boolean, categorical
        'nominal',      # wc_win - boolean, categorical
        'nominal',      # lg_win - boolean, categorical
        'nominal',      # ws_win - boolean, categorical
        'ratio'         # total_playoff_wins - numeric with true zero
    ]
})

COLS


,column_name,data_type,scale
team_id,team_id,object,nominal
year,year,int64,interval
total_salary,total_salary,int64,ratio
wins,wins,int64,ratio
div_win,div_win,bool,nominal
wc_win,wc_win,bool,nominal
lg_win,lg_win,bool,nominal
ws_win,ws_win,bool,nominal
total_playoff_wins,total_playoff_wins,int64,ratio
